<a href="https://colab.research.google.com/github/rajkorde/aispikes/blob/main/supervised_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer
from datasets import load_dataset

MODEL_NAME: str = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
if not hasattr(tokenizer, "modules"):
    tokenizer.modules = lambda: iter([])  # Make it callable

policy_model = AutoModelForCausalLMWithValueHead.from_pretrained(MODEL_NAME)
reference_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

class SimpleRewardModel(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.linear = torch.nn.Linear(policy_model.config.hidden_size, 1)

    def forward(self, input_ids: torch.Tensor) -> torch.Tensor:
        return torch.randn(input_ids.shape[0], 1)

reward_model = SimpleRewardModel()

config = PPOConfig(
    batch_size=2,
    learning_rate=1.41e-5,
    mini_batch_size=1,
    output_dir="./models"
)

dataset = load_dataset("imdb", split="train").select(range(10))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
ppo_trainer = PPOTrainer(
    config,
    policy_model,
    reference_model,
    tokenizer,
    reward_model=reward_model,
    train_dataset=prompts
)

AttributeError: 'NoneType' object has no attribute 'base_model_prefix'

In [ ]:
tokenizer.tokenize("What is happening?")

['What', 'Ġis', 'Ġhappening', '?']

In [ ]:
for epoch in range(3):
    for prompt in prompts:
        try:
            query_tensors = tokenizer(prompt, return_tensors="pt").input_ids.to(policy_model.device)
            response_tensors = policy_model.generate(query_tensors, max_length=50)
            response_text = tokenizer.decode(response_tensors[0], skip_special_tokens=True)
            reward_val = reward_model(query_tensors).item()
            ppo_trainer.step([query_tensors], [response_tensors], [torch.tensor([reward_val])])
            print(f"Prompt: {prompt}\nResponse: {response_text}\nReward: {reward_val}\n")
        except Exception as e:
            print(f"Error processing prompt: {prompt}\nError: {e}")

In [ ]:
config = PPOConfig(**ppo_config)

In [ ]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, reward_model=None, train_dataset=None)

AttributeError: 'AutoModelForCausalLMWithValueHead' object has no attribute 'generation_config'

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import PPOTrainer, AutoModelForCausalLMWithValueHead, PPOConfig
from datasets import load_dataset
import random

# Load a tokenizer
MODEL_NAME = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 needs an explicit pad token

# Load policy model (trainable)
policy_model = AutoModelForCausalLMWithValueHead.from_pretrained(MODEL_NAME)

# Load reference model (non-trainable)
reference_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Define PPO configuration
config = PPOConfig(
    batch_size=2,          # Number of prompts per batch
    learning_rate=1.41e-5, # Learning rate
    mini_batch_size=1,     # For PPO step
    output_dir="./models"
)

In [ ]:
ppo_trainer = PPOTrainer(
    config,
    policy_model,
    reference_model,
    tokenizer,
)

TypeError: PPOTrainer.__init__() missing 2 required positional arguments: 'reward_model' and 'train_dataset'

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from trl import PPOConfig, PPOTrainer, AutoModelForCausalLMWithValueHead
from trl import create_reference_model
from trl.core import generate_with_rewards

# Load tokenizer and model

ImportError: cannot import name 'generate_with_rewards' from 'trl.core' (/usr/local/lib/python3.11/dist-packages/trl/core.py)

In [ ]:
ppo_config = PPOConfig(
    learning_rate=1e-5,
    batch_size=2,
    mini_batch_size=2,
    gradient_accumulation_steps=1,
    seed=42,
    remove_unused_columns=True,
    output_dir="./models"
)

In [ ]:
model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id
    ref_model.config.pad_token_id = ref_model.config.eos_token_id

In [ ]:
dataset = load_dataset("imdb", split="train[:1000]")

In [ ]:
sentiment_pipe = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english",
                         device=0 if torch.cuda.is_available() else -1)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def compute_reward(responses, **kwargs):
    # Get sentiment scores
    reward_outputs = sentiment_pipe(responses)
    rewards = []

    # Compute reward based on sentiment - higher score for positive sentiment
    for output in reward_outputs:
        if output["label"] == "POSITIVE":
            rewards.append(output["score"])
        else:
            rewards.append(1 - output["score"])

    # Return rewards as tensor
    return torch.tensor(rewards)

In [ ]:
# Define a function to prepare the dataset
def prepare_dataset(examples):
    # Tokenize inputs and keep track of input length
    query_tensors = []
    for example in examples["text"]:
        # Truncate to first 100 tokens as a prompt
        prompt = example[:100] + "..."
        query_tensors.append(tokenizer.encode(prompt, return_tensors="pt"))

    return {"query": query_tensors}

In [ ]:
processed_dataset = [prepare_dataset(dataset[i:i+100]) for i in range(0, len(dataset), 100)]
processed_dataset = [item for sublist in processed_dataset for item in sublist["query"]]

In [ ]:
ppo_trainer = PPOTrainer(
    ppo_config,
    model,
    ref_model,
    tokenizer,
    train_dataset=processed_dataset,
)

TypeError: PPOTrainer.__init__() missing 1 required positional argument: 'reward_model'

In [ ]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

dataset = load_dataset("stanfordnlp/imdb", split="train")

training_args = SFTConfig(
    max_seq_length=512,
    output_dir="/tmp",
)
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=training_args,
)


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Converting train dataset to ChatML:   0%|          | 0/25000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/25000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/25000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
dataset[11]

{'text': "I can't believe that those praising this movie herein aren't thinking of some other film. I was prepared for the possibility that this would be awful, but the script (or lack thereof) makes for a film that's also pointless. On the plus side, the general level of craft on the part of the actors and technical crew is quite competent, but when you've got a sow's ear to work with you can't make a silk purse. Ben G fans should stick with just about any other movie he's been in. Dorothy S fans should stick to Galaxina. Peter B fans should stick to Last Picture Show and Target. Fans of cheap laughs at the expense of those who seem to be asking for it should stick to Peter B's amazingly awful book, Killing of the Unicorn.",
 'label': 0}

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rajkorde (signdigits) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,3.429700
1000,3.392700


KeyboardInterrupt: 

In [ ]:
from google.colab import output
output.serve_kernel_port_as_window(9000)

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

In [ ]:
output.serve_kernel_port_as_iframe(port=9000)

<IPython.core.display.Javascript object>